#### 두 개의 파일을 읽어서 다음과 같은 데이터프레임을 만들 것

- 자치구별로 cctv 대수 / 최근증가율 / 인구수 / 외국인 / 고령자 / 외국인비율(%) / 고령자비율(%)

- 최근증가율 :: ( (2018+2019+2020)/~2017년까지의 합 ) * 100

- 구별: index로 만들고 필요없는 내용은 지우고 / 두개 데이터 merge하기 (계/ 합계는 이름 다르니까 inner join 하면 알아서 없어짐)

In [2]:
import numpy as np
import pandas as pd

In [3]:
df1 = pd.read_csv('서울시주민등록인구(구별)통계.txt', sep='\t', skiprows=2, encoding='utf-8')
df1.head(3)

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
0,2021.2/4,합계,"4,405,638","9,795,426","4,756,178","5,039,248","9,565,990","4,649,442","4,916,548","229,436","106,736","122,700",2.17,"1,576,971"
1,2021.2/4,종로구,"73,525","155,106","75,009","80,097","146,029","70,985","75,044","9,077","4,024","5,053",1.99,"27,605"
2,2021.2/4,중구,"62,897","132,259","64,526","67,733","123,016","60,019","62,997","9,243","4,507","4,736",1.96,"23,980"


- 서울시주민등록인구통계 필요한 데이터만 불러오기

In [4]:
# 자치구별로 구분하고, 인구수, 외국인수, 65세이상 고령자인구수 구하기
pop = pd.DataFrame(df1,columns=["자치구","계","계.2","65세이상고령자"])
pop.columns=["자치구","인구수","외국인인구수","고령자인구수"]
pop.head(3)

,자치구,인구수,외국인인구수,고령자인구수
0,합계,"9,795,426","229,436","1,576,971"
1,종로구,"155,106","9,077","27,605"
2,중구,"132,259","9,243","23,980"


- , 없애고 숫자로 바꾸기

In [5]:
for column in pop.columns[1:]:
    pop[column] = pop[column].apply(lambda x: int(x.replace(',','')))
pop.head(3)

,자치구,인구수,외국인인구수,고령자인구수
0,합계,9795426,229436,1576971
1,종로구,155106,9077,27605
2,중구,132259,9243,23980


- 외국인비율(%) , 고령자비율(%) 구하기

In [6]:
pop["외국인비율"] = pop["외국인인구수"] / pop["인구수"] * 100
pop["고령자비율"] = pop["고령자인구수"] / pop["인구수"] * 100
pop.head(3)

,자치구,인구수,외국인인구수,고령자인구수,외국인비율,고령자비율
0,합계,9795426,229436,1576971,2.342277,16.099055
1,종로구,155106,9077,27605,5.852127,17.797506
2,중구,132259,9243,23980,6.988560,18.131091


- cctv 자료 필요한 데이터만 불러오기

In [7]:
cctv = pd.read_csv('서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv',
                    skiprows=1, encoding='euc-kr')
cctv.head(3)


,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,"77,032","7,667","2,200","3,491","4,439","6,582","8,129","9,947","9,876","11,961","11,132","1,608"
1,종로구,"1,772",813,0,0,210,150,1,261,85,9,200,43
2,중 구,"2,333",16,114,87,77,236,240,372,386,155,361,289


- , 없애고 숫자로 바꾸기

In [8]:
for column in cctv.columns[1:]:
    cctv[column] = cctv[column].apply(lambda x: int(x.replace(',','')))
cctv.head(3)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,77032,7667,2200,3491,4439,6582,8129,9947,9876,11961,11132,1608
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43
2,중 구,2333,16,114,87,77,236,240,372,386,155,361,289


- 최근 증가율 구하기

In [9]:
a = (cctv["2018년"] + cctv["2019년"] + cctv["2020년"])
b = cctv["총계"] - cctv["2018년"] - cctv["2019년"] - cctv["2020년"] - cctv["2021년"]
cctv["최근증가율"] = ((a / b) * 100).round(2)
cctv.head(3)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,최근증가율
0,계,77032,7667,2200,3491,4439,6582,8129,9947,9876,11961,11132,1608,77.66
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43,20.49
2,중 구,2333,16,114,87,77,236,240,372,386,155,361,289,78.98


- 필요한 데이터만 정리

In [18]:
cctv_fin = pd.DataFrame(cctv,columns=["구분", "총계","최근증가율"])
cctv_fin.columns=["자치구","cctv대수","최근증가율"]
cctv_fin.head()

,자치구,cctv대수,최근증가율
1,종로구,1772,20.49
2,중 구,2333,78.98
3,용산구,2383,78.64
4,성동구,3602,46.13
5,광진구,2588,112.29


- 서울시 인구 파일은 '중구'고 cctv 파일은 '중 구'여서 바꿔줘야 함

In [11]:
cctv_fin["자치구"][2]= "중구"
cctv_fin.head(3)

/var/folders/l6/zf_y8js55058y519pbd3k9480000gn/T/ipykernel_28710/3348765743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cctv_fin["자치구"][2]= "중구"


,자치구,cctv대수,최근증가율
0,계,77032,77.66
1,종로구,1772,20.49
2,중구,2333,78.98


- 두 데이터프레임 merge (계, 합계 이름 달라서 inner join 하면 총합은 알아서 없어짐)

In [19]:
# Inner join
pop_cctv_data = pd.merge(pop,cctv_fin)
pop_cctv_data

,자치구,인구수,외국인인구수,고령자인구수,외국인비율,고령자비율,cctv대수,최근증가율
0,종로구,155106,9077,27605,5.852127,17.797506,1772,20.49
1,용산구,240665,14287,38884,5.936468,16.156899,2383,78.64
2,성동구,295767,6605,45591,2.233177,15.414499,3602,46.13
3,광진구,355306,12825,50815,3.609565,14.301757,2588,112.29
4,동대문구,352259,14219,61262,4.036519,17.391181,2497,34.85
5,중랑구,394679,4707,70172,1.192615,17.779512,3296,191.07
6,성북구,443206,9548,73568,2.154303,16.599053,3958,94.53
7,강북구,306093,3448,63383,1.126455,20.707105,2462,446.95
8,도봉구,322419,2054,62894,0.637059,19.506915,1629,82.82
9,노원구,520005,4008,86697,0.770762,16.672340,2415,66.38


In [13]:
pop_cctv_data.to_csv('서울시_CCTV_인구_병합데이터.csv')

In [ ]:
# df2['최근증가율(%)'] = ((df2.iloc[:,1:8].sum(axis=1)/df2.iloc[:,8:11].sum(axis=1))*100).round(2)
# df2.tail()
# iloc으로 sum 구하기 가능

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [14]:
# Null data가 있는지 확인
cctv.isna().sum().sum()

# 최근증가율 쉽게 구하기
cctv["최근증가율2"] = ( cctv.iloc[:,9:12].sum(axis=1) / cctv.iloc[:,2:9].sum(axis=1) * 100 ).round(2)

In [15]:
# Selection - 필요한 컬럼 추출
cctv =  cctv[['구분', '총계', '최근증가율']]
cctv.head(3)

# 컬럼명 변경
cctv.rename(columns={'구분':'구별','총계':'cctv대수'})

# 첫번째 행 (계) 삭제
cctv.drop([0], inplace=True)


In [16]:
# method overriding
from sklearn.base import BaseEstimator

class MyClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    def predict(self, X):
        pass
    def score(self, X, y):
        pass

In [17]:
# 구이름에서 공백지우기
cctv.구별 = cctv.자치구.str.replace(' ','')
cctv.head()

# cctv 대수가 많은 자치구 top5
cctv.sort_value('cctv대수', asecending=False).head()

AttributeError: 'DataFrame' object has no attribute '자치구'